# README Demo

- Ubuntu 22.04
- Python 3.10.12
- Cannon EOS R100
- Keigan Motor KM-1S


## Befere Starting

Setup all the hardware and adjust the settings as necessary.

- Serup tripod and turntable
- Connect all devices and power on
- Configure network settings
- Adjust the camera focus

In [1]:
import os
import json
import time
from pprint import pprint

import serial.tools.list_ports
import pandas as pd

from importers import JumpDir
with JumpDir(os.path.join('..', 'src'), os.getcwd()):
	__import__('ugokukun')
from ugokukun.ugoku_kun import UgokuKun
from ugokukun.ugoku_helpers import UgokuHelpers

## Set up directories

In [2]:
config_dir = os.path.join(os.getcwd(), 'conf')
output_dir = os.path.join(os.getcwd(), 'dump')

# Search Serial Port and edit `device_list.json`

**NOTE**: Keigan Motor comes with a Type-C USB cable, but that is only for power. You need to use a Micro USB cable to use USB Serial.

For Linux:

If sudo privileges are required, try adding user to the `dialout` group.

[https://askubuntu.com/questions/133235/how-do-i-allow-non-root-access-to-ttyusb0](https://askubuntu.com/questions/133235/how-do-i-allow-non-root-access-to-ttyusb0)

```bash
sudo adduser $USER dialout
```

In [3]:
ports = serial.tools.list_ports.comports()
for port, desc, hwid in ports:
    print(f"{port}: {desc} [{hwid}]")
print("# of ports:", len(ports))

/dev/ttyUSB0: FT230X Basic UART - FT230X Basic UART [USB VID:PID=0403:6015 SER=DM00KFY6 LOCATION=3-4]
# of ports: 1


Select the port for your turntable.

In [4]:
# Set your port here
# port = '/dev/ttyUSB0'
port = ports[0].device

Open `device_list.json` and manually set port if necessary.

If it seems OK, skip to [next step](#instantiate-ugokukun).

In [5]:
devices_json = UgokuHelpers.import_json(
	os.path.join(config_dir, 'device_list.json')
)
pprint(devices_json)

{'cannon': {'camera_0': '192.168.11.152:8080'},
 'keigan': {'turntable_0': '/dev/ttyUSB0'}}


In [6]:
turntable_id_to_reset = "turntable_0" # Choose your turntable
devices_json["keigan"][turntable_id_to_reset] = port
pprint(devices_json)

{'cannon': {'camera_0': '192.168.11.152:8080'},
 'keigan': {'turntable_0': '/dev/ttyUSB0'}}


Write out `device_list.json`.

In [7]:
with open(os.path.join(config_dir, 'device_list.json'), 'w') as f:
    json.dump(devices_json, f, indent=4)

## Instantiate UgokuKun

First read `camera_setup.csv` to set up the camera. Later, we will load the tasks csv file.

In [8]:
now = time.strftime("%Y%m%d_%H%M%S")
log_fname = os.path.join(output_dir, f"{now}.log")
log_fname

session = UgokuKun(
	task_csv_path    = os.path.join(config_dir, 'camera_setup.csv'),
	device_json_path = os.path.join(config_dir, 'device_list.json'),
	log_path         = log_fname
)

NOTICE: attempting to kill any process using the serial port: /dev/ttyUSB0
  - shell error:  
  - Connecting to motor...
Precheck error: No data received
	attemping to reconnect...


Initialized Keigan motor at: 
  - /dev/ttyUSB0
API URL: http://192.168.11.152:8080/ccapi
  - Request success: 200
  - Connection established
API URL: http://192.168.11.152:8080/ccapi/ver100/deviceinformation
  - Request success: 200
  - Acquired device information
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/settings
  - Request success: 200
  - Current shooting parameters: {"shootingmodedial":{"value":"m"},"av":{"value":"f10","ability":["f2.8","f3.2","f3.5","f4.0","f4.5","f5.0","f5.6","f6.3","f7.1","f8.0","f9.0","f10","f11","f13","f14","f16","f18","f20","f22"]},"tv":{"value":"5\"","ability":["bulb","30\"","25\"","20\"","15\"","13\"","10\"","8\"","6\"","5\"","4\"","3\"2","2\"5","2\"","1\"6","1\"3","1\"","0\"8","0\"6","0\"5","0\"4","0\"3","1\/4","1\/5","1\/6","1\/8","1\/10","1\/13","1\/15","1\/20","1\/25","1\/30","1\/40","1\/50","1\/60","1\/80","1\/100","1\/125","1\/160","1\/200","1\/250","1\/320","1\/400","1\/500","1\/640","1\/800","1\/1000","1\/1250","1\/1600","1\/2000","

...reconnecting...
Probably connected to Keigan motor (not sure until it's used).
   If this is not the first connection to the motor, it can get stuck in an endless loop in:
   - pykeigan.usbcontroller.USBController.reconnect()
   If the connection works fine, you can just ignore it.


### Check devices

In [9]:
print(session.cannon_cameras)
print(session.keigan_motors)

{'camera_0': '192.168.11.152:8080'}
{'turntable_0': '/dev/ttyUSB0'}


### Check task csv

In [10]:
session.csv_df

,task_id,wait_time,target,action,param,payload
0,set_aperture,3,camera_0,aperture,f10,NaN
1,ser_exposure,3,camera_0,exposure,-0_2/3,NaN
2,set_col_temp,3,camera_0,color_temperature,7200,NaN
3,set_iso,3,camera_0,iso,auto,NaN
4,set_shutter_speed,3,camera_0,shutter_speed,"5""",NaN


## Setup camera

After checking that the task csv is correct, we can setup the cameras.

- `UgokuKun.run()` will run the tasks normally.
- `UgokuKun.test_run()` will run the tasks ignoring the wait times

In [11]:
session.test_run()

Executing task id: set_aperture , action: aperture
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/settings/av
  - Request success: 200
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/settings
  - Request success: 200
  - Current shooting parameters: {"shootingmodedial":{"value":"m"},"av":{"value":"f10","ability":["f2.8","f3.2","f3.5","f4.0","f4.5","f5.0","f5.6","f6.3","f7.1","f8.0","f9.0","f10","f11","f13","f14","f16","f18","f20","f22"]},"tv":{"value":"5\"","ability":["bulb","30\"","25\"","20\"","15\"","13\"","10\"","8\"","6\"","5\"","4\"","3\"2","2\"5","2\"","1\"6","1\"3","1\"","0\"8","0\"6","0\"5","0\"4","0\"3","1\/4","1\/5","1\/6","1\/8","1\/10","1\/13","1\/15","1\/20","1\/25","1\/30","1\/40","1\/50","1\/60","1\/80","1\/100","1\/125","1\/160","1\/200","1\/250","1\/320","1\/400","1\/500","1\/640","1\/800","1\/1000","1\/1250","1\/1600","1\/2000","1\/2500","1\/3200","1\/4000"]},"iso":{"value":"auto","ability":["auto","100","125","160","200","250","320","400","500",

After running above, check the images taken by the camera.

If it needs reconfiguration, edit the `camera_setup.csv` and reload the tasks csv file.

In [12]:
session.load_task_csv(os.path.join(config_dir, 'camera_setup.csv'))

,task_id,wait_time,target,action,param,payload
0,set_aperture,3,camera_0,aperture,f10,NaN
1,ser_exposure,3,camera_0,exposure,-0_2/3,NaN
2,set_col_temp,3,camera_0,color_temperature,7200,NaN
3,set_iso,3,camera_0,iso,auto,NaN
4,set_shutter_speed,3,camera_0,shutter_speed,"5""",NaN


If the loaded tasks csv file is correct, re-run `UgokuKun.test_run()`.

In [13]:
session.test_run()

Executing task id: set_aperture , action: aperture
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/settings/av
  - Request success: 200
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/settings
  - Request success: 200
  - Current shooting parameters: {"shootingmodedial":{"value":"m"},"av":{"value":"f10","ability":["f2.8","f3.2","f3.5","f4.0","f4.5","f5.0","f5.6","f6.3","f7.1","f8.0","f9.0","f10","f11","f13","f14","f16","f18","f20","f22"]},"tv":{"value":"5\"","ability":["bulb","30\"","25\"","20\"","15\"","13\"","10\"","8\"","6\"","5\"","4\"","3\"2","2\"5","2\"","1\"6","1\"3","1\"","0\"8","0\"6","0\"5","0\"4","0\"3","1\/4","1\/5","1\/6","1\/8","1\/10","1\/13","1\/15","1\/20","1\/25","1\/30","1\/40","1\/50","1\/60","1\/80","1\/100","1\/125","1\/160","1\/200","1\/250","1\/320","1\/400","1\/500","1\/640","1\/800","1\/1000","1\/1250","1\/1600","1\/2000","1\/2500","1\/3200","1\/4000"]},"iso":{"value":"auto","ability":["auto","100","125","160","200","250","320","400","500",

Repeat this process until the camera is set up correctly.

## Run the tasks

In order to run the actual tasks, load the appropriate tasks csv file and run `UgokuKun.run()`.

In [14]:
actual_task = os.path.join(config_dir, 'as_you_wish.csv')
session.load_task_csv(actual_task)

,task_id,wait_time,target,action,param,payload
0,shutter_0,3,camera_0,shutter,False,NaN
1,cw_0,3,turntable_0,cw,5,NaN
2,shutter_5,3,camera_0,shutter,False,NaN
3,cw_5,3,turntable_0,cw,5,NaN
4,shutter_10,3,camera_0,shutter,False,NaN
...,...,...,...,...,...,...
141,cw_350,3,turntable_0,cw,5,NaN
142,shutter_355,3,camera_0,shutter,False,NaN
143,cw_355,3,turntable_0,cw,5,NaN
144,shutter_360,3,camera_0,shutter,False,NaN


If the tasks csv file is correct, run `UgokuKun.run()`.

In [15]:
session.run()

Executing task id: shutter_0 , action: shutter
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/control/shutterbutton
  - Request success: 200
  - Shutter button pressed
Executing task id: cw_0 , action: cw
Motor turned -5 degrees clockwise at 30 rpm.
Executing task id: shutter_5 , action: shutter
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/control/shutterbutton
  - Request success: 200
  - Shutter button pressed
Executing task id: cw_5 , action: cw
Motor turned -5 degrees clockwise at 30 rpm.
Executing task id: shutter_10 , action: shutter
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/control/shutterbutton
  - Request success: 200
  - Shutter button pressed
Executing task id: cw_10 , action: cw
Motor turned -5 degrees clockwise at 30 rpm.
Executing task id: shutter_15 , action: shutter
API URL: http://192.168.11.152:8080/ccapi/ver100/shooting/control/shutterbutton
  - Request success: 200
  - Shutter button pressed
Executing task id: cw_15 , action: 

## TODO dump camera cofig

Forgot to implement `CannonWrapper.dump_attribures()` in `UgokuKun`.

In [17]:
for camera_id in session.cannon_cameras.keys():
    session.cannon_instances[camera_id].dump_attributes(os.path.join(output_dir, f"{camera_id}.json"))

  - Settings dumped to: /home/ugoku/Dropbox (Danlab)/personal/naro-intern/ugoku-kun/readme_demo/dump/camera_0.json
